<a href="https://colab.research.google.com/github/cu-applied-math/SciML-Class/blob/main/Labs/lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp
import scipy.optimize as opt

# Lab 8

We'll consider the [logistic equation](https://en.wikipedia.org/wiki/Logistic_function#In_ecology:_modeling_population_growth) for describing popluation growth.
$y(t)$ represents the population at time $t$, and satisifies the equation
$$dy/dt = \theta y \left( 1 - \frac{y}{k} \right)$$
where $\theta$ is the **growth rate** (often referred to as "r" but we'll use $\theta$ to match other conventions), and $k$ is a carrying capacity of the population.

This is a very simple first-order ODE, covered in any good undergrad ODE class, and has a closed-form solution,
$$y(t) = \frac{k}{ 1 + \frac{k-y_0}{y_0}e^{-\theta t}}$$
if $y(0) = y_0$ is the initial condition.

**Motivation for our lab**
We're observing a population of organisms and we know they evolve according to the logistic equation, and we know that they have a given carrying capacity of $k=10$.  We observe two data points:

| time | |  population |
| :--- | --- | :--- |
| $t_0$ |0 | $y_0=1$ |
| $t_1$| 2 | $y_1 = 7$ |

and our scientific question is, **what was the growth rate $\theta$**?

Our lab is divided into several parts:
- Part 1: familiarity with scipy's ODE solver
  - This should be quick
- Part 2 and 3a: work things out by hand
  - This should be pen-and-paper, and should also be fairly quick
- Part 3b: adjoint state method
  - This may take longer. This is the heart of the lab
- Part 4: root-finding
  - This is "extra". Only do this if you finish part 3b

**What to turn in for credit?**
- If you were in class for the lab, just turn in the plots from parts 1a and 1b
- If you were not in class during the lab day, then please spend 50 minutes on the lab and turn in your complete ipynb file (exported to PDF) showing your work up to at least question 3b. You can use the lab solutions but your notebook should not be identical to the solutions.

## Part 1a: solve with an ODE solver
We want to plot the function and see what it looks like. Let's solve it with `scipy.integrate.solve_ivp` ([documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)), starting at $t=0$ with a value of $y_0 = 1$, and using $k=10$. Solve until $t=5$ using $\theta=3$ as our "guess" for the growth rate for now.

In [ ]:
k = 10

def f(t,y,theta):
    """ Logistic equation right-hand-side
    dy/dt = theta y (1-y/k),  y(t) is the population, theta is growth rate parameter, k is karrying kapacity
    """
    raise NotImplementedError()  # You should implement this

y0 = 1.
T  = 5
t_span = (0,T)
theta = 3

soln = solve_ivp(f,t_span, [y0], dense_output=True, args=(theta,))

tGrid = np.linspace( 0, T, num=100 )
t1 = 2.
y1 = 7.
plt.plot( tGrid, soln['sol']( tGrid ).flatten() )
plt.plot( [0,T], [y1,y1], 'k:' )
plt.plot( [t1,t1], [0,k], 'k--' )
plt.xlabel('time')
plt.title(f'theta is {theta}')
plt.show()

## Part 1b: solve backward with an ODE solver

Using the final condition from part 1a (i.e., the value of $y$ at $T=5$), use the ODE solver to solve the same ODE still on the interval $[0,T]$ except give it the "final condition" rather than the initial condition.

Plot both forward and backward solutions and verify they are about the same.  *Tip: you may want to adjust `atol` and/or `rtol` in the ODE solver to get increased accuracy*

**Turn in this plot (to Canvas) with forward and backward solutions** to get credit for the lab

In [ ]:
# ... todo ...

## Part 2: find the right answer

Solving problems is always easier if you know the answer before you start.  In our case, we want to know what $\theta$ is. Because we have a closed-form solution for the ODE solution $y$, you can work this out by hand.

What is the correct value of $\theta$ that generates the observed data?

In [ ]:
# ... todo, analytic calculation ...

## Part 3a: find the derivative "by hand"

Let our loss function be $\ell(\theta) = y(t_1)$, i.e., our "loss" $L$ as a function of $y(t_1)$ is simply the identity map.   We want $\frac{d\ell}{d\theta}$

(Why do we want this? Because then we could do root-finding on $\ell(\theta) = y_1$ and use Newton's method)

- Work out what $\frac{d\ell}{d\theta}$ is by hand (since we have the analytic form of $y$).

In [ ]:
# ... todo, analytic calculation ...

## Part 3b: find the derivative via the adjoint state method

For more complicated ODEs without closed-form solutions, we may not be able to find the derivative "by hand", so we could use the adjoint state method.  

The point of today's lab is to practice that.

Below is the full adjoint sensitivities "Algorithm 2" from Appendix C from [Neural Ordinary Differential Equations](https://arxiv.org/abs/1806.07366) by Chen, Rubanova, Bettencourt and Duvenaud (NeurIPS 2018), converted to our notation ($y$ instead of $z$) and ignoring irrelevant features:

- Input dynamics parameters $\theta$, start time $t_0$, stop time $t_1$, final state $y(t_1)$, loss gradient $\frac{\partial L}{\partial y(t_1)}$
- Define initial augmented state $s_0 = [y(t_1)), \frac{\partial L}{\partial y(t_1)}, 0]$
  - The augmented variable is $s=[y,a^{(y)}, a^{(\theta)}]$ and $a$ are the adjoint quantities, $a^{(y)}(t) = \frac{dL}{dy(t)}$ and $a^{(\theta)} = \frac{dL}{d\theta}$
  - Our goal is to find $a^{(\theta)}$, e.g., at $t=t_0$, since $\theta$ is constant
  - define the fuction `augmented_dynamics`$(t,s,\theta)$
    - $s=[y,a^{(y)}, a^{(\theta)}]$
    - return $\frac{ds}{dt} = f(y(t), t, \theta), -a^{(y)}(t) \frac{\partial f}{\partial y}, -a^{(y)}(t) \frac{\partial f}{\partial \theta}]$
  - call `ODE_solver`$(s_0,$ `augmented_dynamics`$, t_1,t_0, \theta)$


Your **task** is to define appropriate dynamics and initial states to pass in to the ODE solver and solve, and then extracting the derviative. You can check with the results from part 3a

In [ ]:
# ... todo ...

## Part 4: root-finding to find $\theta$

### Part 4a: derivative-free method
Our unknown parameter $\theta$ is 1D, and 1D root-finding is fairly easy, even if we do not know the derivative.  Our rules-of-thumb from earlier labs is that not knowing the derivative for optimization problems is not a big deal in small dimensions, and a similar rule-of-thumb applies for root finding.

So, using a `scipy.optimize` [scalar root-finding](https://docs.scipy.org/doc/scipy/reference/optimize.html#scalar-functions) algorithm that doesn't require a derivative.
- You could do brute-force (grid search) which isn't so bad in 1D
- You can do bisection methods, or fancier variants in the `scipy.optimize` library (`root_scalar`, `brentq`, `brenth`, `bisect`, etc.).

### Part 4b: root-finding with Newton (aka Newton-Raphson)
Since we do have the derivative information (either via the adjoint-state method or using your analytic calculation), we can use Newton-Raphson (also implemented in `scipy.optimize` as `newton`).  So **use Newton-Raphson** to find $\theta$.

Note: In 1D, the main advantage of Newton-Raphson compared to bisection or other derivative-free methods is that it quickly gives you extremely high accuracy.  
- You may not always need extremely high accuracy
- The accuracy might be limited by the accuracy of your function evaluation or derivative evaluation, i.e., the accuracy of the ODE solve. If we don't solve the ODE to more than 6 digits, we shouldn't expect to find $\theta$ to 10 digits...

In [ ]:
# ... todo, use (4a) a derivative-free method, and then (4b) Newton-Raphson